In [ ]:
%pip install pmdarima

Importing Libraries and Data:

In [ ]:
# Importing Basic libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Importing time series specific libraries
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.ar_model import AR
from statsmodels.tsa.arima.model import ARIMA
import pmdarima as pm
from pmdarima import auto_arima
from statsmodels.tsa.statespace import sarimax
from statsmodels.tsa.stattools import kpss

# Libaraies for evaluation of model
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, median_absolute_error, mean_squared_log_error
from statsmodels.tsa.arima.model import ARIMAResults

Data Set:

In [ ]:
# Read CSV file
data = pd.read_csv('productsales.csv')
data
data.describe()

Data Exploration and Preprocessing:

In [ ]:
data['date'] = pd.to_datetime(data['date'], infer_datetime_format=True)
data = data.set_index(['date'])
print(data.head())
# Ensure sorted by date
data = data.sort_values(by='date')

In [ ]:
# Total sales over time
total_sales  = data.groupby('date')['number_sold'].sum()
total_sales.describe(include = 'all')

In [ ]:
# Calculate total sales by product type
product_sales = data.groupby('product_type')['number_sold'].sum()

# Identify the most and least sold product types
most_sold_product_type = product_sales.idxmax()
least_sold_product_type = product_sales.idxmin()

# Calculate total sales by store (stall id)
store_sales = data.groupby('stall_id')['number_sold'].sum()

# Identify the store with the most and least action figures sold
store_most_sold = store_sales.idxmax()
store_least_sold = store_sales.idxmin()

# Calculate average number sold per product type to see typical sales volume
avg_sales_per_product = data.groupby('product_type')['number_sold'].mean()

# Count how many sales transactions each store has to understand store activity level
store_transaction_counts = data.groupby('stall_id').size()

# Output results
print(f"Most sold product type: {most_sold_product_type}")
print(f"Least sold product type: {least_sold_product_type}")
print(f"Store with the most action figures sold: {store_most_sold}")
print(f"Store with the least action figures sold: {store_least_sold}")
print("\nAverage number sold per product type:")
print(avg_sales_per_product)
print("\nNumber of sales transactions per store:")
print(store_transaction_counts)

Data Set Plot:

In [ ]:
# Plot total sales over time
plt.figure(figsize=(16, 8))
plt.plot(total_sales)
plt.title('Total Sales Over Time')
plt.xlabel('Date')
plt.ylabel('Total Number Sold')
plt.grid(True)
plt.show()

Visualization of ACF AND PACF:

In [ ]:
from statsmodels.tsa.stattools import acf, pacf

ts = pd.DataFrame(total_sales)
values = ts['number_sold']

# Calculate ACF and PACF values
acf_vals = acf(values, nlags=30)
pacf_vals = pacf(values, nlags=30)

plt.figure(figsize=(10,6))

plt.subplot(211)
plt.bar(range(len(acf_vals)), acf_vals, color='skyblue')
plt.title('ACF Bar Plot')
plt.xlabel('Lag')
plt.ylabel('Autocorrelation')

plt.subplot(212)
plt.bar(range(len(pacf_vals)), pacf_vals, color='lightgreen')
plt.title('PACF Bar Plot')
plt.xlabel('Lag')
plt.ylabel('Partial Autocorrelation')

plt.tight_layout()
plt.show()

Check for Stationarity:

In [ ]:
# Option 1
# Results for Augmented Dickey-Fuller (ADF) Test
print("Results of Dickey-Fuller Test:")

adf_stat = adf_result[0]
adf_pvalue = adf_result[1]
adf_lags = adf_result[2]
adf_nobs = adf_result[3]
adf_crit = adf_result[4]

print(f"{'Test Statistic':<25} {adf_stat:.6f}")
print(f"{'p-value':<25} {adf_pvalue:.6f}")
print(f"{'#Lags Used':<25} {adf_lags}")
print(f"{'Number of Observations Used':<25} {adf_nobs}")

for key in ['1%', '5%', '10%']:
    print(f"Critical Value ({key}){'':<10} {adf_crit[key]:.6f}")

print("\nResults of KPSS Test:")

kpss_stat = kpss_result[0]
kpss_pvalue = kpss_result[1]
kpss_lags = kpss_result[2]
kpss_crit = kpss_result[3]

print(f"{'Test Statistic':<25} {kpss_stat:.6f}")
print(f"{'p-value':<25} {kpss_pvalue:.6f}")
print(f"{'Lags Used':<25} {kpss_lags}")

for key in ['10%', '5%', '2.5%', '1%']:
    print(f"Critical Value ({key}){'':<10} {kpss_crit[key]:.6f}")

#Option 2:
# # Perform Augmented Dickey-Fuller (ADF) test
# adf_result = adfuller(total_sales)
# print(f"ADF Statistic: {adf_result[0]:.6f}")
# print(f"p-value: {adf_result[1]:.6f}")
# print("Critical Values:")
# for key, value in adf_result[4].items():
#     print(f"\t{key}: {value:.3f}")
#
# # Perform KPSS test
# kpss_result = kpss(total_sales, regression='c')
# print(f"\nKPSS Statistic: {kpss_result[0]:.6f}")
# print(f"p-value: {kpss_result[1]:.6f}")
# print("Critical Values:")
# for key, value in kpss_result[3].items():
#     print(f"\t{key}: {value:.3f}")


Differencing:

In [ ]:
#Option 1:
# Apply first differencing
first_difference = total_sales.diff().dropna()

# Results for ADF Test on First Differenced Series
print("Results of Dickey-Fuller Test (First Difference):")

adf_stat_diff = adf_result_diff[0]
adf_pvalue_diff = adf_result_diff[1]
adf_lags_diff = adf_result_diff[2]
adf_nobs_diff = adf_result_diff[3]
adf_crit_diff = adf_result_diff[4]

print(f"{'Test Statistic':<35} {adf_stat_diff:.6f}")
print(f"{'p-value':<35} {adf_pvalue_diff:.6f}")
print(f"{'#Lags Used':<35} {adf_lags_diff}")
print(f"{'Number of Observations Used':<35} {adf_nobs_diff}")
for key in ['1%', '5%', '10%']:
    print(f"Critical Value ({key}){'':<20} {adf_crit_diff[key]:.6f}")
print("dtype: float64\n")

# Results for KPSS Test on First Differenced Series
print("Results of KPSS Test (First Difference):")

kpss_stat_diff = kpss_result_diff[0]
kpss_pvalue_diff = kpss_result_diff[1]
kpss_lags_diff = kpss_result_diff[2]
kpss_crit_diff = kpss_result_diff[3]

print(f"{'Test Statistic':<35} {kpss_stat_diff:.6f}")
print(f"{'p-value':<35} {kpss_pvalue_diff:.6f}")
print(f"{'Lags Used':<35} {kpss_lags_diff}")
for key in ['10%', '5%', '2.5%', '1%']:
    print(f"Critical Value ({key}){'':<20} {kpss_crit_diff[key]:.6f}")
print("dtype: float64")

# Option 2:
# # Apply first differencing
# first_difference = total_sales.diff().dropna()
#
# # Apply ADF test on first differenced series
# adf_result_diff = adfuller(first_difference)
# print('ADF Statistic (First Difference): %f' % adf_result_diff[0])
# print('p-value: %f' % adf_result_diff[1])
# print('Critical Values:')
# for key, value in adf_result_diff[4].items():
#     print('\t%s: %.3f' % (key, value))
#
# # Apply KPSS test on first differenced series
# kpss_result_diff = kpss(first_difference, regression='c')
# print('\nKPSS Statistic (First Difference): %f' % kpss_result_diff[0])
# print('p-value: %f' % kpss_result_diff[1])
# print('Critical Values:')
# for key, value in kpss_result_diff[3].items():
#     print('\t%s: %.3f' % (key, value))

Checking the Plot of the differencing:

In [ ]:
plt.figure(figsize=(12, 6))

plt.subplot(121)
plot_acf(first_difference, lags=40, ax=plt.gca())
plt.title('ACF of First Differenced Series')

plt.subplot(122)
plot_pacf(first_difference, lags=40, ax=plt.gca())
plt.title('PACF of First Differenced Series')

plt.show()

Auto-Arima:

In [ ]:
# Split the data into training and testing sets based on the specified years
train_data = total_sales[(total_sales.index >= '2010-01-01') & (total_sales.index <= '2018-12-31')]
test_data = total_sales[(total_sales.index >= '2019-01-01') & (total_sales.index <= '2019-12-31')]

# Fit the auto ARIMA model
model = pm.auto_arima(y=train_data)
results = model.fit(y=train_data)
print(results.summary())

# Make predictions on the training set (adjusting for differencing)
start_index = model.order[1]
train_predict = results.predict_in_sample(start=start_index, end=len(train_data)-1)
train_predict_series = pd.Series(train_predict, index=train_data.index[start_index:])

# Make predictions on the test set
predictions = results.predict(n_periods=len(test_data))
predictions_series = pd.Series(predictions, index=test_data.index)

Plot Result:

In [ ]:
# Plot the results with transparency
plt.figure(figsize=(15, 7))
plt.plot(total_sales.index, total_sales, label='Original Time Series', color='teal')
plt.plot(train_predict_series.index, train_predict_series, label='Fitted Values', color='navy', linewidth=1)
plt.plot(predictions_series.index, predictions_series, label='Forecasted Values', color='coral', linewidth=2)
plt.axvline(train_data.index[-1], color='red', linestyle='--', label='Train/Test Split')
plt.legend()
plt.title('ARIMA Model - Fitted and Forecasted Values')
plt.xlabel('Date')
plt.ylabel('Sales')
plt.xticks(rotation=45)
plt.grid(True)
plt.show()

Evaluation of the Model:

In [ ]:
# Align lengths of train_data and train_predict_series
train_data_aligned = train_data[start_index:]

# ================================
# Evaluation Metrics - Training Data
# ================================
print("Evaluation Results for Training Data (Auto-ARIMA):")

# Mean Squared Error
MSE_train = mean_squared_error(train_data_aligned, train_predict_series)
print(f"{'Mean Squared Error (MSE)':<40} = {MSE_train:.6f}")

# Mean Absolute Error
MAE_train = mean_absolute_error(train_data_aligned, train_predict_series)
print(f"{'Mean Absolute Error (MAE)':<40} = {MAE_train:.6f}")

# Root Mean Squared Error
RMSE_train = np.sqrt(MSE_train)
print(f"{'Root Mean Squared Error (RMSE)':<40} = {RMSE_train:.6f}")

# ================================
# Evaluation Metrics - Testing Data
# ================================
print("\nEvaluation Results for Testing Data (Auto-ARIMA):")

# Mean Squared Error
MSE_test = mean_squared_error(test_data, predictions_series)
print(f"{'Mean Squared Error (MSE)':<40} = {MSE_test:.6f}")

# Mean Absolute Error
MAE_test = mean_absolute_error(test_data, predictions_series)
print(f"{'Mean Absolute Error (MAE)':<40} = {MAE_test:.6f}")

# Root Mean Squared Error
RMSE_test = np.sqrt(MSE_test)
print(f"{'Root Mean Squared Error (RMSE)':<40} = {RMSE_test:.6f}")
